# Modules and data

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/JacopoMalatesta/imdb_most_popular_films/main/data/cleaned_df.csv',
            index_col = 0)

# Putting the data on the cloud

I created a MYSQL database on Amazon RDS by following this tutorial: https://aws.amazon.com/getting-started/hands-on/create-mysql-db/

Let's split the dataframe into three. 

In [3]:
films = df[['id', 'title', 'release_date', 'country', 'language', 'genre', 'studios', 'color', 'aspect_ratio', 'last_updated']]

stats = df[['id', 'runtime', 'budget', 'revenue', 'imdb_rating', 'imdb_rating_count', 'metascore', 'user_review_count',
           'critic_review_count']]

people = df[['id', 'director', 'writer', 'actors', 'cinematographer', 'editor', 'composer', 'production_designer', 
             'art_director','costume_designer', 'producers']]

Let's create the engine 

In [4]:
username = os.environ.get('rds_mysql_imdb_username')
password = os.environ.get('rds_mysql_imdb_psw')
host = os.environ.get('rds_mysql_imdb_host')
dbname = 'mysql_imdb'

connection = create_engine(f'mysql://{username}:{password}@{host}/{dbname}').connect()

Let's upload our three dataframes

In [5]:
films.to_sql(con = connection, name = 'films', if_exists = 'replace', chunksize = 500, index = False)
stats.to_sql(con = connection, name = 'stats', if_exists = 'replace', chunksize = 500, index = False)
people.to_sql(con = connection, name = 'people', if_exists = 'replace', chunksize = 500, index = False)

Let's run a simple to query to see if everything looks ok

In [6]:
query = ''' SELECT title, EXTRACT(YEAR FROM CAST(release_date AS DATE)) year, imdb_rating_count
FROM films
JOIN people USING (id)
JOIN stats USING (id)
WHERE director LIKE '%Kar-wai%'
ORDER BY 3 DESC

'''

pd.read_sql_query(sql = query, con = connection)

,title,year,imdb_rating_count
0,In the Mood for Love,2000,139801
1,Chungking Express,1994,73772
2,2046,2004,55827
3,My Blueberry Nights,2007,53563
4,Fallen Angels,1995,34318
5,The Grandmaster,2013,33458


# Postgresql

I've created a PostgreSQL database on RDS as well.

Let's connect to our database

In [7]:
username = os.environ.get('rds_postgres_imdb_username')
password = os.environ.get('rds_postgres_imdb_psw')
host = os.environ.get('rds_postgres_imdb_host')
dbname = 'postgres_imdb'

postgres_connection = create_engine(f'postgresql://{username}:{password}@{host}/{dbname}').connect()

Uploading our three dataframes

In [9]:
films.to_sql(con = postgres_connection, name = 'films', if_exists = 'replace', chunksize = 100, index = False)
stats.to_sql(con = postgres_connection, name = 'stats', if_exists = 'replace', chunksize = 100, index = False)
people.to_sql(con = postgres_connection, name = 'people', if_exists = 'replace', chunksize = 100, index = False)

Running a simple query to test things out

In [10]:
query = ''' SELECT title, EXTRACT(YEAR FROM CAST(release_date AS DATE)) year, imdb_rating_count
FROM films
JOIN people USING (id)
JOIN stats USING (id)
WHERE director LIKE '%John Carpenter%'
ORDER BY 3 DESC

'''

pd.read_sql_query(sql = query, con = connection)

,title,year,imdb_rating_count
0,The Thing,1982,399248
1,Halloween,1978,260419
2,Escape from New York,1981,136234
3,Big Trouble in Little China,1986,133634
4,They Live,1988,122574
5,Christine,1983,78691
6,Escape from L.A.,1996,73071
7,The Fog,1980,70657
8,In the Mouth of Madness,1995,66455
9,Vampires,1998,56839
